In [69]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from collections import defaultdict

In [65]:
with open("varia.xml", "r", encoding="utf-8") as file:
    # Read each line in the file, readlines() returns a list of lines
    content = file.readlines()
    # Combine the lines in the list into a string
    content = "".join(content)
    soup = bs(content, "xml")

In [70]:
#d = defaultdict(list)

docDates,regDates,mscontentscount,biblcount = 0,0,0,0
i=0

for item in soup.findAll("TEI"):
    itemDate = "N/A"
    itemType = item['subtype']
    if itemType == "dedication" or itemType == "greeting":
        print(i)
        xmlid = item['xml:id']
        #print(item["xml:id"])
        try:
            bibl = item.find("bibl")
            title = bibl.find("title")
            #print("Title in bibl element")
            biblcount += 1
        except:
            bibl = item.find("msContents")
            title = bibl.find("title")
            #print("Title in msContents element")
            mscontentscount += 1
        for match in title.findAll():
            match.unwrap()
        #item = soup.find("TEI")
        #itemHead = item.find("subtype")
        itemDate = item.find("docDate")
        try:
            xmldate = itemDate['when']
            docDates+=1
            #print(xmlid,title,item['subtype'],itemDate['when'],"(docDate), ID:")
        except:
            itemDate = item.find("date")
            xmldate = itemDate['when']
            regDates+=1
            #removeNumbers = itemDate.replace("-","")
            #stringToReplace = "V"+str(removeNumbers)
            #xmlid = xmlid.replace(stringToReplace,"")
        datenumbers = xmldate.replace("-","")
        if "Vudat" in xmlid:
            persid = xmlid.replace("Vudat","")
        else:
            persid = xmlid[1:] # don't remove all Vs lol
        persid = persid.replace(datenumbers,"")
        print(xmlid,title,item['subtype'],itemDate['when'],", pe/orgID:",persid)
        print("\n")
        i+=1
print(i,"docDate elements:",docDates,"\nregular date elements:",regDates,"\nmscontentscount:",mscontentscount,"\nbibl count:",biblcount)

0
V18500415CLD <title type="main">Hilsen i Christopher L. Dues stambok</title> greeting 1850-04-15 , pe/orgID: CLD


1
V18560226NN <title type="main">Dedikasjon til ukjent mottaker i Gildet paa Solhoug</title> dedication 1856-02-26 , pe/orgID: NN


2
V1856HJ <title type="main">Dedikasjon til Herman Johnson i Gildet paa Solhoug</title> dedication 1856 , pe/orgID: HJ


3
V18571004MB <title>Dedikasjon til Michael Birkeland i Fru
                     Inger til Østeraad</title> dedication 1857-10-04 , pe/orgID: MB


4
V185712JCG <title type="main">Dedikasjon til J. Chr. Gamborg i Fru Inger til Østeraad</title> dedication 1857-12 , pe/orgID: JCG


5
V1858kongO2 <title>Dedikasjon til prins Oscar (senere Oscar II) i Hærmændene paa Helgeland</title> dedication 1858 , pe/orgID: kongO2


6
V1858CRU <title>Dedikasjon til Carl Richard Unger i Hærmændene paa Helgeland</title> dedication 1858 , pe/orgID: CRU


7
V1858KK <title type="main">Dedikasjon til Knud Knudsen i Hærmændene paa Helgeland</title>

In [62]:
item

<TEI subtype="art" type="varia" xml:id="VudatFossen_tegn">

Variadokument
O.L. Mohr: Ibsen som maler, nr. 27.
På baksiden lapp med attestering ved Francis Bull (25.05.1941)


Oslo
Norsk Folkemuseum/Ibsenmuseet
NF.2006-0053


Maleri av Vøringsfossen.
Henrik Ibsen
Udatert

Egenhendig manuskript. Akvarell. 19,5 x 15 cm. 



Blekk. Henrik Ibsen. Hovedhånd


03.12.2019when-atributtLa in "årstallet" 9999 i when-atributt for sortering
    







</TEI>